## 單純用BERT去預測一個句子的label，在句子的sep token後面再加上aspect term token (CLS+text+SEP+aspect+SEP)，測試unique input (可以看做那句子是只屬於對應aspect)對aspect term polarity準確度

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

### 對處理好的laptop、restaurant的train、test資料作前處理

In [2]:
laptop_train = pd.read_csv('dataset/laptop_train_processed.csv', encoding='utf-8')
restaurant_train = pd.read_csv('dataset/restaurant_train_processed.csv', encoding='utf-8')
laptop_test = pd.read_csv('dataset/laptop_test_processed.csv', encoding='utf-8')
restaurant_test = pd.read_csv('dataset/restaurant_test_processed.csv', encoding='utf-8')

# 把train的資料串在一起且多加一個aspect
train_data = laptop_train.append(restaurant_train)
train_data = train_data.reset_index(drop=True)

# 把test的資料串在一起且多加一個aspect
test_data = laptop_test.append(restaurant_test)
test_data = test_data.reset_index(drop=True)

print('訓練資料數:', len(train_data))
print('筆電測試資料數:', len(laptop_test))
print('餐廳測試資料數:', len(restaurant_test))
print('測試資料數:', len(test_data))
train_data.tail(10)

訓練資料數: 5915
筆電測試資料數: 638
餐廳測試資料數: 1120
測試資料數: 1758


,text,aspect,polarity
5905,"From the appetizers we ate, the dim sum and ot...",appetizers,positive
5906,"From the appetizers we ate, the dim sum and ot...",dim sum,positive
5907,"From the appetizers we ate, the dim sum and ot...",foods,positive
5908,"From the appetizers we ate, the dim sum and ot...",food,positive
5909,Each table has a pot of boiling water sunken i...,table,neutral
5910,Each table has a pot of boiling water sunken i...,pot of boiling water,neutral
5911,Each table has a pot of boiling water sunken i...,meats,neutral
5912,Each table has a pot of boiling water sunken i...,vegetables,neutral
5913,Each table has a pot of boiling water sunken i...,rice,neutral
5914,Each table has a pot of boiling water sunken i...,glass noodles,neutral


In [3]:
#把polarity變成數字的label，positive是2，neutral是1，negative是0
train_data.loc[train_data['polarity']=='positive', 'label'] = 2
train_data.loc[train_data['polarity']=='negative', 'label'] = 0
train_data.loc[train_data['polarity']=='neutral', 'label'] = 1
train_data['label'] = train_data['label'].astype(int)

test_data.loc[test_data['polarity']=='positive', 'label'] = 2
test_data.loc[test_data['polarity']=='negative', 'label'] = 0
test_data.loc[test_data['polarity']=='neutral', 'label'] = 1
test_data['label'] = test_data['label'].astype(int)

test_data.head(10)

,text,aspect,polarity,label
0,"Boot time is super fast, around anywhere from ...",Boot time,positive,2
1,tech support would not fix the problem unless ...,tech support,negative,0
2,Set up was easy.,Set up,positive,2
3,Did not enjoy the new Windows 8 and touchscree...,Windows 8,negative,0
4,Did not enjoy the new Windows 8 and touchscree...,touchscreen functions,negative,0
5,Other than not being a fan of click pads (indu...,internal speakers,negative,0
6,Other than not being a fan of click pads (indu...,price tag,positive,2
7,Other than not being a fan of click pads (indu...,click pads,negative,0
8,No installation disk (DVD) is included.,installation disk (DVD),neutral,1
9,"It's fast, light, and simple to use.",use,positive,2


### Bert資料前處理

In [4]:
import tensorflow as tf
from transformers import BertTokenizer, BertModel, TFBertForSequenceClassification, TFBertModel

In [5]:
# Load pre-trained model tokenizer, to convert our text into tokens that correspond to BERT’s vocabulary.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
# 把aspect term token串在sep後面的練習
text = 'aspect term token'
aspect = 'aspect term'
text_tok = tokenizer.tokenize(text) # 把文字變成token
aspect_tok = tokenizer.tokenize(aspect)
text_id = tokenizer.convert_tokens_to_ids(text_tok) # 把token變成Id
aspect_id = tokenizer.convert_tokens_to_ids(aspect_tok)
print(text_id)
print(aspect_id)
text_cls_sep = tokenizer.build_inputs_with_special_tokens(text_id) # 加入CLS、SEP token id
print(text_cls_sep)
text_sep_aspect = text_cls_sep + aspect_id
print(text_sep_aspect)

[7814, 2744, 19204]
[7814, 2744]
[101, 7814, 2744, 19204, 102]
[101, 7814, 2744, 19204, 102, 7814, 2744]


#### 找出單句最多token

In [7]:
# 找出最多text add aspect中最多是幾個token，不包含CLS跟SEP
def find_max_token(pd):
    max_token = 0
    for i in range(len(pd)):
        text = pd.loc[i, 'text']
        aspect = pd.loc[i, 'aspect']
        text_aspect = text + aspect
        tokens_len = len(tokenizer.tokenize(text_aspect))
        if tokens_len>max_token:
            max_token = tokens_len
    return max_token

In [8]:
# 找出text add aspect中token最多的是幾個token，不包含CLS跟SEP
train_max_token = find_max_token(train_data)
test_max_token = find_max_token(test_data)
print('訓練資料集token最多是:', train_max_token)
print('測試資料集token最多是:', test_max_token)

訓練資料集token最多是: 91
測試資料集token最多是: 99


### 正式把資料轉換成token(padding)

#### 把句子轉變成token(CLS+text+SEP+asepct+SEP)+(padding)的function

In [9]:
# 把維度固定在128維
input_dim = 128
def input_ids_all(pd):
    pd['input_ids'] = 'N/A'
    for i in range(len(pd)):
        text = pd.loc[i, 'text']
        aspect = pd.loc[i, 'aspect']
        text_tokens = tokenizer.tokenize(text) # 把text轉成token
        aspect_tokens = tokenizer.tokenize(aspect) # 把aspect轉成token
        
        text_input_ids = tokenizer.convert_tokens_to_ids(text_tokens) # 把text token轉成text token id
        aspect_input_ids = tokenizer.convert_tokens_to_ids(aspect_tokens) # 把aspect token轉成aspect token id
        
        text_input_ids_cls = tokenizer.build_inputs_with_special_tokens(text_input_ids) # aspect token id加上CLS、SEP token id
        input_ids = text_input_ids_cls + aspect_input_ids # 把aspect token id接在text token id 後面 (CLS+text+SEP+aspect)
        input_ids.append(102)
        input_ids = np.array(input_ids)
        
        if len(input_ids) < input_dim:
            n = input_dim - len(input_ids)
            input_ids = np.pad(input_ids, (0, n), mode ='constant', constant_values=(0)) # array右邊append n 個 0  補長度到512
        
        pd['input_ids'][i] = input_ids
    return pd

In [10]:
# 將text轉成token，後面加上aspect token存進dataframe
train_data = input_ids_all(train_data)
test_data = input_ids_all(test_data)

/home/daniel/anaconda3/envs/bertenv/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#### 把句子轉變成token(CLS+text+SEP+asepct)+(padding)的function

In [11]:
# 把維度固定在128維
input_dim = 128
def input_ids_all_2(text, aspect):
    text_tokens = tokenizer.tokenize(text) # 把text轉成token
    aspect_tokens = tokenizer.tokenize(aspect) # 把aspect轉成token
        
    text_input_ids = tokenizer.convert_tokens_to_ids(text_tokens) # 把text token轉成text token id
    aspect_input_ids = tokenizer.convert_tokens_to_ids(aspect_tokens) # 把aspect token轉成aspect token id
        
    text_input_ids_cls = tokenizer.build_inputs_with_special_tokens(text_input_ids) # aspect token id加上CLS、SEP token id
    input_ids = text_input_ids_cls + aspect_input_ids # 把aspect token id接在text token id 後面 (CLS+text+SEP+aspect)
    input_ids.append(102)
    input_ids = np.array(input_ids)
        
    if len(input_ids) < input_dim:
        n = input_dim - len(input_ids)
        input_ids = np.pad(input_ids, (0, n), mode ='constant', constant_values=(0)) # array右邊append n 個 0  補長度到512
    return input_ids

In [12]:
train_data['input_ids_2'] = train_data.apply(lambda column: input_ids_all_2(column['text'], column['aspect']), axis=1)
test_data['input_ids_2'] = test_data.apply(lambda column: input_ids_all_2(column['text'], column['aspect']), axis=1)

In [13]:
n = 5111
print(train_data.loc[n, 'text'])
print(train_data.loc[n, 'aspect'])
print(train_data.loc[n, 'input_ids'])
print(train_data.loc[n, 'input_ids_2'])

The place is sleek, modern and playfull and i will return again frequently.
place
[  101  1996  2173  2003 21185  1010  2715  1998 18378  2140  1998  1045
  2097  2709  2153  4703  1012   102  2173   102     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
[  101  1996  2173  2003 21185  1010  2715  1998 18378  2140  1998  1045
  2097  2709  2153  4703  1012   102  2173   102     0     0     

In [14]:
n = 509
print(test_data.loc[n, 'text'])
print(test_data.loc[n, 'aspect'])
print(test_data.loc[n, 'input_ids'])
print(test_data.loc[n, 'input_ids_2'])

But I wanted the Pro for the CD/DVD player.
CD/DVD player
[ 101 2021 1045 2359 1996 4013 2005 1996 3729 1013 4966 2447 1012  102
 3729 1013 4966 2447  102    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[ 101 2021 1045 2359 1996 4013 2005 1996 3729 1013 4966 2447 1012  102
 3729 1013 4966 2447  102    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0  

In [15]:
# 把train data的input_ids提出存進list
train_input_ids = list()
for i in range(len(train_data)):
    np_id = train_data.loc[i, 'input_ids']
    train_input_ids.append(np_id)
train_input_ids = np.array(train_input_ids)
print(train_input_ids.shape)
train_input_ids

(5915, 128)


array([[ 101, 1045, 3715, ...,    0,    0,    0],
       [ 101, 1045, 3715, ...,    0,    0,    0],
       [ 101, 1996, 6627, ...,    0,    0,    0],
       ...,
       [ 101, 2169, 2795, ...,    0,    0,    0],
       [ 101, 2169, 2795, ...,    0,    0,    0],
       [ 101, 2169, 2795, ...,    0,    0,    0]])

In [16]:
# test data的input_ids提出存進list
test_input_ids = list()
for i in range(len(test_data)):
    np_id = test_data.loc[i, 'input_ids']
    test_input_ids.append(np_id)
test_input_ids = np.array(test_input_ids)
print(test_input_ids.shape)
test_input_ids

(1758, 128)


array([[  101,  9573,  2051, ...,     0,     0,     0],
       [  101,  6627,  2490, ...,     0,     0,     0],
       [  101,  2275,  2039, ...,     0,     0,     0],
       ...,
       [  101, 24519, 10439, ...,     0,     0,     0],
       [  101, 24519, 10439, ...,     0,     0,     0],
       [  101, 24519, 10439, ...,     0,     0,     0]])

In [17]:
# 把train data label變成numpy
train_label = train_data['label'].to_numpy()
print(len(train_label))
train_label

5915


array([1, 2, 0, ..., 1, 1, 1])

In [18]:
# 把test data label變成numpy
test_label = test_data['label'].to_numpy()
print(len(test_label))
test_label

1758


array([2, 0, 2, ..., 2, 2, 2])

In [19]:
# 檢查polarity跟label有沒有不一樣
print('test_data')
for i in range(15):
    print(test_data.loc[i, 'polarity'], test_data.loc[i, 'label'], test_label[i])

test_data
positive 2 2
negative 0 0
positive 2 2
negative 0 0
negative 0 0
negative 0 0
positive 2 2
negative 0 0
neutral 1 1
positive 2 2
positive 2 2
positive 2 2
positive 2 2
positive 2 2
positive 2 2


### Model

In [27]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule 
# num_labels=3 分3類
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
model.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2307      
Total params: 109,484,547
Trainable params: 109,484,547
Non-trainable params: 0
_________________________________________________________________


In [28]:
model_fit = model.fit(train_input_ids, train_label, 
                      batch_size=4, epochs=4, 
                      validation_data=(test_input_ids, test_label))

Train on 5915 samples, validate on 1758 samples
Epoch 1/4
5915/5915 [==============================] - 144s 24ms/sample - loss: 0.6917 - accuracy: 0.7187 - val_loss: 0.5860 - val_accuracy: 0.7497
Epoch 2/4
5915/5915 [==============================] - 130s 22ms/sample - loss: 0.4406 - accuracy: 0.8326 - val_loss: 0.4966 - val_accuracy: 0.8100
Epoch 3/4
5915/5915 [==============================] - 130s 22ms/sample - loss: 0.2955 - accuracy: 0.8898 - val_loss: 0.4807 - val_accuracy: 0.8197
Epoch 4/4
5915/5915 [==============================] - 130s 22ms/sample - loss: 0.1943 - accuracy: 0.9349 - val_loss: 0.5324 - val_accuracy: 0.8203


### 看confusion matrix

In [29]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [30]:
# test data confusion
predictions_test= model.predict(test_input_ids) # 輸出的是n*5的編碼值array
predictions_test = np.argmax(predictions_test, axis=1) # axis = 1是取行的最大值的索引，0是列的最大值的索引
print(accuracy_score(test_label, predictions_test))
print(confusion_matrix(test_label, predictions_test))
print(classification_report(test_label, predictions_test))

0.820250284414107
[[285  23  16]
 [ 91 191  83]
 [ 68  35 966]]
              precision    recall  f1-score   support

           0       0.64      0.88      0.74       324
           1       0.77      0.52      0.62       365
           2       0.91      0.90      0.91      1069

    accuracy                           0.82      1758
   macro avg       0.77      0.77      0.76      1758
weighted avg       0.83      0.82      0.82      1758



In [31]:
# laptop_test confusion
laptop_test_input_ids = test_input_ids[:638]
laptop_test_label = test_label[:638]
predictions_lap_test = model.predict(laptop_test_input_ids)
predictions_lap_test = np.argmax(predictions_lap_test, axis=1) # axis = 1是取行的最大值的索引，0是列的最大值的索引
print(accuracy_score(laptop_test_label, predictions_lap_test))
print(confusion_matrix(laptop_test_label, predictions_lap_test))
print(classification_report(laptop_test_label, predictions_lap_test))

0.7617554858934169
[[118   6   4]
 [ 63  76  30]
 [ 31  18 292]]
              precision    recall  f1-score   support

           0       0.56      0.92      0.69       128
           1       0.76      0.45      0.57       169
           2       0.90      0.86      0.88       341

    accuracy                           0.76       638
   macro avg       0.74      0.74      0.71       638
weighted avg       0.79      0.76      0.76       638



In [32]:
# laptop_test confusion
restaurant_test_input_ids = test_input_ids[638:]
restaurant_test_label = test_label[638:]
predictions_res_test = model.predict(restaurant_test_input_ids)
predictions_res_test = np.argmax(predictions_res_test, axis=1) # axis = 1是取行的最大值的索引，0是列的最大值的索引
print(accuracy_score(restaurant_test_label, predictions_res_test))
print(confusion_matrix(restaurant_test_label, predictions_res_test))
print(classification_report(restaurant_test_label, predictions_res_test))

0.8535714285714285
[[167  17  12]
 [ 28 115  53]
 [ 37  17 674]]
              precision    recall  f1-score   support

           0       0.72      0.85      0.78       196
           1       0.77      0.59      0.67       196
           2       0.91      0.93      0.92       728

    accuracy                           0.85      1120
   macro avg       0.80      0.79      0.79      1120
weighted avg       0.85      0.85      0.85      1120

